In [0]:
%pip install python-dotenv openai azure-cosmos pypdf

In [0]:
dbutils.library.restartPython()

In [0]:
import sys
import os
from pathlib import Path

# ==== プロジェクトルートを Python パスに追加 ====

# 例: /Workspace/Repos/sohei.kurita@microsoft.com/jsai2025-pdf-processor
project_root = Path("/Workspace/Repos/soheikurita@microsoft.com/demo-resonac-databricks-script")

if not project_root.exists():
    raise RuntimeError(f"project_root が存在しません: {project_root}\n実際のパスに書き換えてください。")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("✓ project_root を sys.path に追加しました:", project_root)

# ==== カレントディレクトリをプロジェクトルートに変更（.env などの相対パス用） ====
os.chdir(project_root)
print("✓ カレントディレクトリ:", os.getcwd())

# ==== Jupyter/Databricks 上での asyncio 対応 ====
import nest_asyncio
nest_asyncio.apply()
print("✓ nest_asyncio を適用しました")

# ==== STAGE 環境変数（ライブラリ側で .env 読み込みなどに使っている想定） ====
os.environ.setdefault("STAGE", "local")
print("✓ STAGE:", os.environ["STAGE"])

# ==== ライブラリのインポート ====
from demo.adapter.library import PdfProcessorClient

print("✓ PdfProcessorClient をインポートしました")


In [0]:
from pathlib import Path

# ==== PdfProcessorClient の初期化 ====
client = PdfProcessorClient(
    deployment_name="gpt-5-mini",          
    embedding_deployment_name="text-embedding-3-small",  
    show_progress=True,
)

print("✓ PdfProcessorClient を初期化しました")

# ==== DBFS パスをローカルパス (/dbfs/...) に変換するヘルパ ====

def dbfs_to_local_path(dbfs_path: str) -> str:
    """
    'dbfs:/silver/... -> /dbfs/silver/...' に変換
    """
    if dbfs_path.startswith("dbfs:"):
        return dbfs_path.replace("dbfs:", "/dbfs", 1)
    return dbfs_path


# 入力 PDF（Bronze 層）
PDF_DIR_DBFS = "dbfs:/bronze/jsai2025/pdfs"
PDF_DIR_LOCAL = dbfs_to_local_path(PDF_DIR_DBFS)

# 出力 JSON / メタデータ（Silver 層）
SILVER_OUTPUT_DBFS = "dbfs:/silver/jsai2025/pdf_processed"
SILVER_OUTPUT_LOCAL = dbfs_to_local_path(SILVER_OUTPUT_DBFS)

# Silver 出力ディレクトリを作成
Path(SILVER_OUTPUT_LOCAL).mkdir(parents=True, exist_ok=True)

print("PDF_DIR_DBFS         :", PDF_DIR_DBFS)
print("PDF_DIR_LOCAL        :", PDF_DIR_LOCAL)
print("SILVER_OUTPUT_DBFS   :", SILVER_OUTPUT_DBFS)
print("SILVER_OUTPUT_LOCAL  :", SILVER_OUTPUT_LOCAL)


In [0]:
from pathlib import Path

# ローカル (/dbfs/...) 経由で PDF を 1 件だけ取得
pdf_dir = Path(PDF_DIR_LOCAL)
pdf_files = sorted(pdf_dir.glob("*.pdf"))

if not pdf_files:
    raise RuntimeError(f"PDF が見つかりません: {pdf_dir}")

test_pdf = pdf_files[0]
print(f"テスト対象 PDF: {test_pdf}")

original_paper, search_data, paper_file, search_data_file = client.process_and_save_to_json(
    pdf_file_path=str(test_pdf),
    output_dir=SILVER_OUTPUT_LOCAL,
)

print("\n✓ 処理が完了しました")
print(f"  論文ID: {original_paper.id}")
print(f"  Title: {original_paper.title}")
print(f"  Total chunks: {len(search_data)}")
print(f"  paper_file: {paper_file}")
print(f"  search_data_file: {search_data_file}")


In [0]:
from pyspark.sql.functions import input_file_name

silver_path = "dbfs:/silver/jsai2025/pdf_processed"

# JSON を Spark DataFrame として読み込む
df = (
    spark.read
        .option("multiline", "true")
        .json(silver_path)
        .withColumn("file_name", input_file_name())
)

print("行数:", df.count())
display(df.limit(5))


In [0]:
from pyspark.sql import functions as F

# binaryFile ソースで PDF 一覧を取得
pdf_df = (
    spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.pdf")
    .load(PDF_DIR_DBFS + "/*")
    .select("path")  # path 列だけ使う
)

print("PDF ファイル件数:", pdf_df.count())
display(pdf_df.limit(10))


In [0]:
from demo.adapter.library import PdfProcessorClient

def create_client():
    client = PdfProcessorClient(
        deployment_name="gpt-5-mini",
        embedding_deployment_name="text-embedding-3-small",
        show_progress=False,   
    )
    return client


In [0]:
from pyspark.sql import Row

def process_one_pdf(pdf_path: str):

    client = create_client()

    try:
        original_paper, search_data, paper_file, search_data_file = client.process_and_save_to_json(
            pdf_file_path=pdf_path,
            output_dir=SILVER_OUTPUT_LOCAL,
        )

        return [Row(
            pdf_path=pdf_path,
            paper_file=paper_file,
            search_data_file=search_data_file,
            status="ok",
            error=None,
        )]

    except Exception as e:
        return [Row(
            pdf_path=pdf_path,
            paper_file=None,
            search_data_file=None,
            status="error",
            error=str(e),
        )]


In [0]:
result_df = pdf_rdd.flatMap(process_one_pdf).toDF()
display(result_df)

